<a href="https://colab.research.google.com/github/Yarida-nes/weatherwise-Yarida-Kaewthong/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [5]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai


In [6]:
import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

Enter your API key: Perth


## 📦 Setup and Configuration
Import required packages and setup environment.

In [10]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
# ✅ Import after installing (if needed)
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

# Add any other setup code here

In [8]:
!pip install pyinputplus

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyinputplus: filename=pyinputplus-0.2.12-py3-none-any.whl size=11348 sha256=51cd692f7b5d782393e92c89ee3927d16e7c32659238e6ac83584354acbd2e43
  Stored in directory: /root/.cache/pip/wheels/85/81/c1/325c8199c5c4de8a0b9af900ff9cd8bd69853e3d9419493512
  Created wheel for pysimplevalidate: filename=pysimplevalidate-0.2.12-py3-none-any.whl size=16225 sha256=e5655f28edaf6643f9ac38b19933495b6f1c1602fdbe4d95b65d4f55e56d355f
  Stored in directory: /root/.cache/pip/wheels/09/de/18/4dd2be0dbdf31870c6f0abde43b0de197b1c095655ad7da5c2
  Created wheel for stdiomask: filename=stdiomask

## 🌤️ Weather Data Functions

In [19]:
# Define get_weather_data() function here
def get_weather_data(location, forecast_days=5):
    """
    Retrieve weather data for a specified location.

    Args:
        location (str): City or location name
        forecast_days (int): Number of days to forecast (1-5)

    Returns:
        dict: Weather data including current conditions and forecast
    """
    try:
        data = get_weather(location)  # fetch data

        # Handle case where no data is returned
        if not data:
            return {"error": "Location not found"}

        # Access data as a dictionary
        data_dict = data.dict()

        # Keep only requested forecast days
        data_dict["forecast"] = data_dict.get("forecast", [])[:forecast_days]
        return data_dict

    except Exception as e:
        return {"error": str(e)}

## 📊 Visualisation Functions

In [63]:
# Define create_temperature_visualisation() here
def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of temperature data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    forecast = weather_data.get("forecast", ["Perth"])
    if not forecast:   # do nothing if no forecast
        return

    days = [day["date"] for day in weather_data["forecast"]]
    max_temps = [int(day["maxtempC"]) for day in weather_data["forecast"]]
    min_temps = [int(day["mintempC"]) for day in weather_data["forecast"]]

    location = weather_data.get("location", "Perth")

    fig, ax = plt.subplots()
    ax.plot(days, max_temps, label="Max Temp (°C)", marker="o", color="red")
    ax.plot(days, min_temps, label="Min Temp (°C)", marker="o", color="blue")
    ax.set_title(f"Temperature Forecast – {location}")
    ax.set_xlabel("Date")
    ax.set_ylabel("°C")
    ax.legend()

    if output_type == 'figure':
        return fig
    else:
        plt.show()

In [64]:
def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of precipitation data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    forecast = weather_data.get("forecast", [])
    if not forecast:   # do nothing if no forecast
        return


    days = [day["date"] for day in weather_data["forecast"]]

    # pick the first hourly block’s chance of rain for each day
    precip = [int(day["hourly"][0]["chanceofrain"]) for day in weather_data["forecast"]]

    location = weather_data.get("location", "Unknown")

    fig, ax = plt.subplots()
    ax.bar(days, precip, color="skyblue")
    ax.set_title(f"Chance of Rain – {location}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Chance of Rain (%)")

    if output_type == 'figure':
        return fig
    else:
        plt.show()

In [67]:
sample = get_weather_data("Perth", forecast_days=3)
print(sample)

{'current_condition': [{'FeelsLikeC': '28', 'FeelsLikeF': '83', 'cloudcover': '0', 'humidity': '14', 'localObsDateTime': '2025-09-24 02:17 PM', 'observation_time': '06:17 AM', 'precipInches': '0.0', 'precipMM': '0.0', 'pressure': '1022', 'pressureInches': '30', 'temp_C': '29', 'temp_F': '84', 'uvIndex': '7', 'visibility': '10', 'visibilityMiles': '6', 'weatherCode': '113', 'weatherDesc': [{'value': 'Sunny'}], 'weatherIconUrl': [{'value': ''}], 'winddir16Point': 'NNW', 'winddirDegree': '336', 'windspeedKmph': '12', 'windspeedMiles': '8'}], 'nearest_area': [{'areaName': [{'value': 'Maylands'}], 'country': [{'value': 'Australia'}], 'latitude': '-31.933', 'longitude': '115.883', 'population': '10447', 'region': [{'value': 'Western Australia'}], 'weatherUrl': [{'value': ''}]}], 'request': [{'query': 'Lat -31.95 and Lon 115.86', 'type': 'LatLon'}], 'weather': [{'astronomy': [{'moon_illumination': '4', 'moon_phase': 'Waxing Crescent', 'moonrise': '07:01 AM', 'moonset': '08:34 PM', 'sunrise': 

/tmp/ipython-input-70675569.py:21: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  data_dict = data.dict()


In [76]:
sample = get_weather_data("Perth", forecast_days=3)
print("Forecast data:", sample.get("forecast"))
if sample.get("forecast"):
    create_temperature_visualisation(sample)


Forecast data: []


/tmp/ipython-input-70675569.py:21: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  data_dict = data.dict()


## 🤖 Natural Language Processing

In [ ]:
# Define parse_weather_question() and generate_weather_response() here
def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: Extracted information including location, time period, and weather attribute
    """
    pass

## 🧭 User Interface

In [ ]:
# Define menu functions using pyinputplus or ipywidgets here

## 🧩 Main Application Logic

In [ ]:
# Tie everything together here
def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    pass

## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.